# Linea de Muerte Z

##  1. Big Picture

* Preprocesamiento
   * Se quitan del datataset los *envenenados* atributos  mprestamos_personales y cprestamos_personales
   * Se agregan lags y delta_lags de orden 1 y 2
* Modelado no se optimizan hiperarámetros
* Produccion
   * Entrenamieento final
      * Se entrena en {202101, 202102, 202103, 202104}
      * Se hace un conservador undersampling de **0.50** de los "CONTINUA"
      * POS = {"BAJA+1", "BAJA+2"}
      * librería  *zLightGBM*
         * **100** canaritos se agregan al comienzo del dataset, porque ahora hay mas datos
         * gradient_bound se deja en su default de  **0.1**
   * Clasificacion
      * Se corta en 11000  envios

---
Resultados :
* ganancia de 391.085 M en el Private Leaderboard ( 11.000 en el Public )
* utiliza 17 GB de memoria RAM
* corre en 30 minutos

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 14:38:58 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658747,35.2,1454648,77.7,1228049,65.6
Vcells,1223008,9.4,8388608,64.0,1975175,15.1


In [2]:
PARAM <- list()
PARAM$experimento <- "zmuerte-50"
PARAM$semilla_primigenia <- 654821

## Preprocesamiento

### Generacion de la clase_ternaria

In [3]:
Sys.time()
#install.packages("data.table")
library(data.table)
Sys.time()

[1] "2025-11-16 14:39:04 UTC"

[1] "2025-11-16 14:39:04 UTC"

In [4]:
Sys.time()
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )
Sys.time()

[1] "2025-11-16 14:39:06 UTC"

[1] "2025-11-16 14:39:30 UTC"

In [5]:
Sys.time()
# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 14:39:30 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,767116,41.0,1454648,77.7,1454648,77.7
Vcells,722144829,5509.6,1017394257,7762.2,846135486,6455.6


[1] "2025-11-16 14:39:32 UTC"

### Eliminacion de Features

La auténtica  **salsa mágica**  de este script es la eliminación de estos dos atributos del dataset ya que tran problemas con el Data Drifting
* mprestamos_personales
* cprestamos_personales

el problema con estos campos se detectó manualmente mediante un analisis exploratorio de datos y análisis de corridas de LightGBM en meses previos.

In [6]:
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]
Sys.time()

[1] "2025-11-16 14:39:32 UTC"

### Feature Engineering Historico

A partir del dataset crudo, se genera el campo  clase_ternaria

In [7]:
Sys.time()
print("Empieza el feature engineering histórico")
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()
print("Terminó lo otro y ahora vamos con variables que podrían resultar interesantes")
serv_cols <- c(
  "mcaja_ahorro","mcaja_ahorro_adicional",
  "mcuentas_saldo",
  "mtarjeta_visa_consumo","mtarjeta_master_consumo",
  "mpayroll","mpayroll2",
  "mtransferencias_recibidas","mtransferencias_emitidas"
)

serv_exist <- intersect(serv_cols, names(dataset))

dataset[, (paste0(serv_exist, "_on")) :=
          lapply(.SD, function(x) as.integer(!is.na(x) & x > 0)),
        .SDcols = serv_exist]

dataset[, organs_on := rowSums(.SD, na.rm = TRUE),
        .SDcols = paste0(serv_exist, "_on")]

setorder(dataset, numero_de_cliente, foto_mes)
dataset[, organs_on_lag1 := shift(organs_on, 1, NA, "lag"), by = numero_de_cliente]
dataset[, organs_on_lag2 := shift(organs_on, 2, NA, "lag"), by = numero_de_cliente]
dataset[, organs_drop1 := organs_on_lag1 - organs_on]
dataset[, organs_drop2 := organs_on_lag2 - organs_on]


dataset[, mpayroll_lag1 := shift(mpayroll, 1, NA, "lag"), by = numero_de_cliente]
dataset[, pierde_payroll :=
          as.integer(!is.na(mpayroll_lag1) & mpayroll_lag1 > 0 &
                       (is.na(mpayroll) | mpayroll == 0))]

dataset[, mcuentas_saldo_lag1 := shift(mcuentas_saldo, 1, NA, "lag"), by = numero_de_cliente]
dataset[, saldo_se_fue :=
          as.integer(!is.na(mcuentas_saldo_lag1) & mcuentas_saldo_lag1 > 0 &
                       (is.na(mcuentas_saldo) | mcuentas_saldo == 0))]

dataset[, visa_trx_lag1 := shift(ctarjeta_visa_transacciones, 1, NA, "lag"), by = numero_de_cliente]
dataset[, apaga_visa :=
          as.integer(!is.na(visa_trx_lag1) & visa_trx_lag1 > 0 &
                       (is.na(ctarjeta_visa_transacciones) | ctarjeta_visa_transacciones == 0))]

dataset[, hubo_mov := as.integer(
  ifelse(is.na(ctrx_quarter), 0, ctrx_quarter) > 0 |
  ifelse(is.na(ctarjeta_visa_transacciones), 0, ctarjeta_visa_transacciones) > 0 |
  ifelse(is.na(ctarjeta_debito_transacciones), 0, ctarjeta_debito_transacciones) > 0 |
  ifelse(is.na(mtransferencias_emitidas), 0, mtransferencias_emitidas) > 0 |
  ifelse(is.na(mtransferencias_recibidas), 0, mtransferencias_recibidas) > 0
)]


dataset[, meses_sin_mov := {
  r <- integer(.N)
  conta <- 0L
  for (i in seq_len(.N)) {
    if (hubo_mov[i] == 0L) {
      conta <- conta + 1L
    } else {
      conta <- 0L
    }
    r[i] <- conta
  }
  r
}, by = numero_de_cliente]

vars_sensibles <- c("mcuentas_saldo", "ctrx_quarter", "mtarjeta_visa_consumo")
vars_sensibles <- intersect(vars_sensibles, names(dataset))

for (v in vars_sensibles) {
  dataset[, paste0(v, "_lag1") := shift(get(v), 1, NA, "lag"), by = numero_de_cliente]
  dataset[, paste0(v, "_delta1") := get(v) - get(paste0(v, "_lag1"))]
}


sens_delta_cols  <- paste0(vars_sensibles, "_delta1")
sens_delta_exist <- intersect(sens_delta_cols, names(dataset))

if (length(sens_delta_exist) == 0) {
  dataset[, apagado_sensible := 0L]
} else if (length(sens_delta_exist) == 1) {
  unico <- sens_delta_exist[1]
  dataset[, apagado_sensible :=
            as.integer(!is.na(get(unico)) & get(unico) < 0)]
} else {
  dataset[, apagado_sensible :=
            rowSums(as.data.frame(lapply(.SD, function(x)
              as.integer(!is.na(x) & x < 0))), na.rm = TRUE),
          .SDcols = sens_delta_exist]
}

dataset[, organs_on_diff := organs_on - shift(organs_on, 1, NA, "lag"), by = numero_de_cliente]

dataset[, streak_baja_organs := {
  r <- integer(.N)
  conta <- 0L
  for (i in seq_len(.N)) {
    if (!is.na(organs_on_diff[i]) && organs_on_diff[i] < 0) {
      conta <- conta + 1L
    } else {
      conta <- 0L
    }
    r[i] <- conta
  }
  r
}, by = numero_de_cliente]

dataset[, muerte_score :=
          1.5 * pierde_payroll +
          1.0 * saldo_se_fue +
          1.0 * (meses_sin_mov >= 2) +
          0.5 * (apagado_sensible >= 2) +
          0.5 * (streak_baja_organs >= 2)
]
Sys.time()
print("Terminó esta otra parte y ahora finalizamos con slopes")
library(data.table)

setDT(dataset)
setorder(dataset, numero_de_cliente, foto_mes)

vars_num <- c(
  "ctrx_quarter","cpayroll_trx","mpayroll","mpasivos_margen","mcaja_ahorro",
  "mtarjeta_visa_consumo","mcuentas_saldo","mautoservicio",
  "ctarjeta_visa_transacciones","ctarjeta_debito_transacciones",
  "Visa_msaldopesos","Visa_msaldototal","mrentabilidad_annual",
  "ccomisiones_mantenimiento","mtransferencias_recibidas",
  "mcomisiones_mantenimiento","Visa_mpagominimo","mextraccion_autoservicio"
)

vars_num <- intersect(vars_num, names(dataset))

f_slope <- function(y, t) {
  ok <- !is.na(y)
  if (sum(ok) < 2) return(NA_real_)
  coef(lm(y[ok] ~ t[ok]))[2]
}

ultimos <- 4  

slopes_lastN <- dataset[
  ,
  {
    idx <- tail(.I, ultimos)              
    t   <- as.numeric(foto_mes[idx])    
    lapply(.SD[idx], f_slope, t = t)    
  },
  by = numero_de_cliente,
  .SDcols = vars_num
]

for (v in vars_num) {
  setnames(slopes_lastN, v, paste0(v, "_slope_last", ultimos))
}

dataset <- merge(
  dataset,
  slopes_lastN,
  by = "numero_de_cliente",
  all.x = TRUE,
  sort = FALSE
)
print("Terminó ese proceso entero")
print("El datase tiene el tamaño siguiente:")
length(dataset)

[1] "2025-11-16 14:39:32 UTC"

[1] "Empieza el feature engineering histórico"


[1] "2025-11-16 14:41:23 UTC"

[1] "Terminó lo otro y ahora vamos con variables que podrían resultar interesantes"


[1] "2025-11-16 14:44:46 UTC"

[1] "2025-11-16 14:44:46 UTC"

[1] "Terminó esta otra parte y ahora finalizamos con slopes"
[1] "Terminó ese proceso entero"
[1] "El datase tiene el tamaño siguiente:"


[1] 795

## Modelado

In [8]:
# training y future
Sys.time()
print("Vamos a entrenar con todos los meses salvo con 202108")
PARAM$train_final$meses <- c(201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106, 202107)
print("El undersampling sería de 0.5%")
PARAM$train_final$undersampling <- 0.05
print("Y el futuro a predecir es 202108")
PARAM$future <- c(202108)

[1] "2025-11-16 14:47:58 UTC"

[1] "Vamos a entrenar con todos los meses salvo con 202108"
[1] "El undersampling sería de 0.5%"
[1] "Y el futuro a predecir es 202108"


### Final Training Strategy

Se entrena en los cuatro meses {202101, 202102, 202103, 202104}
<br>Se hace un conservador undersampling del 50% de la clase mayoritaria (los "CONTINUA" )

In [9]:
print("dataset_tran_final corresponde a los meses que son parte del entrenamiento")
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

[1] "dataset_tran_final corresponde a los meses que son parte del entrenamiento"


In [10]:
Sys.time()
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]
dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L]
dataset_train_final[, azar:= NULL]

[1] "2025-11-16 14:48:43 UTC"

### Target Engineering

In [11]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40
print("Binarizamos clase ternaria")
dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

[1] "Binarizamos clase ternaria"


### Final Model

In [12]:
Sys.time()
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

[1] "2025-11-16 14:49:19 UTC"

Loading required package: zlightgbm



[1] "2025-11-16 14:49:20 UTC"

In [13]:
Sys.time()
PARAM$qcanaritos <- 5

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-11-16 14:49:29 UTC"

[1] "2025-11-16 14:49:30 UTC"

In [14]:
# los campos que se van a utilizar
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [15]:
# dejo los datos en el formato que necesita LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 265735 columnas 799 


[1] "2025-11-16 14:49:57 UTC"

In [16]:
print("acá tengo 799 porque sumé canaritos y volé clase ternaria")

[1] "acá tengo 799 porque sumé canaritos y volé clase ternaria"


Nuevos hiperparámetros en  *zLightGBM*
* canaritos , entero >=0,  cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
* gradient_bound , numero real, >=0  cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo

En el caso que  canaritos==0  y  gradient_bound==0  entonces  xLightGBM se comporta exactamente igual a  LightGBM

Tener en cuenta que el hiperparámetro  min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
<br> realmente valdria la pena experimentar con distintos valores de  min_data_in_leaf, hay potencial de mejora !

In [17]:
# definicion de parametros, los viejos y los nuevos
Sys.time()
PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 20L,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 1.0,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.1  # default de zLightGBM
)

Sys.time()

[1] "2025-11-16 14:51:17 UTC"

[1] "2025-11-16 14:51:17 UTC"

####  Entrenamiento del modelo

Aqui se ejecuta  *zLightGBM*
<br> no se utilizan hiperparámetros optimos para controlar el overfitting
<br> zLightGBM  sabe cuando no hacer un split
<br> si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina
<br>
<br> claramente el  min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [18]:
# entreno el modelo
Sys.time()
modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm)

Sys.time()

[1] "2025-11-16 14:51:27 UTC"

[1] "2025-11-16 15:24:02 UTC"

In [19]:
Sys.time()
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
lgb.save(modelo_final, file="zmodelo.txt")

# grabo un dataset que tiene el detalle de los arboles de LightGBM
tb_arboles <- lgb.model.dt.tree(modelo_final)
fwrite(tb_arboles, file="tb_arboles.txt", sep="\t")

cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
cat("summary de las hojas de los arboles")
summary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])

Sys.time()

[1] "2025-11-16 15:24:02 UTC"

cantidad arbolitos= 1771 
summary de las hojas de los arboles

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    2.0   623.5   999.0   795.1   999.0   999.0 

[1] "2025-11-16 15:28:43 UTC"

In [20]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

In [21]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_pred <- dfuture[, .(numero_de_cliente)]
tb_pred[, prob := prediccion ]
setorder(tb_pred, -prob)
corte <- 11000 
tb_submit <- tb_pred[1:corte, .(numero_de_cliente)]

archivo_salida <- sprintf("KA_%s.csv",
                          format(Sys.time(), "%Y%m%d_%H%M%S"))

fwrite(
  tb_submit,
  file      = archivo_salida,
  sep       = ",",
  col.names = FALSE, 
  row.names = FALSE,
  quote     = FALSE
)

In [22]:
Sys.time()
print("Me armo la predicción así veo de combinarlo")
tb_prediccion1 <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion1[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion1,
  file= "prediccion_paracombinar.txt",
  sep= "\t"
)

[1] "2025-11-16 15:40:36 UTC"

[1] "Me armo la predicción así veo de combinarlo"


In [ ]:
Sys.time()

## Ahora vamos con el LIGHT GBM normalito con ensemble de 5 o 10

Esto debería empezar posterior al FE comprobada la cantidad de columnas. Ah, y canaritos.

In [ ]:
# training y future
Sys.time()
print("Vamos a entrenar con todos los meses salvo con 202108")
PARAM$train_final$meses <- c(201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106, 202107)
print("El undersampling sería de 0.4%")
print("Lo reduzco un poco por una cuestión de tiempo")
PARAM$train_final$undersampling <- 0.04
print("Y el futuro a predecir es 202108")
PARAM$future <- c(202108)

In [ ]:
Sys.time()
print("Estos son los hiperparámetros del LightGBM. Los optimicé en un momento previo y me mató la máquina y no guardé el resultado")

PARAM$train_final <- list()

PARAM$train_final$param_mejores <- list(
  objective            = "binary",
  metric               = "custom",
  first_metric_only    = TRUE,
  boost_from_average   = TRUE,
  feature_pre_filter   = FALSE,
  verbosity            = -100,
  force_row_wise       = TRUE,
  seed                 = 974411,  
  extra_trees          = FALSE,
  max_depth            = -1,
  min_gain_to_split    = 0.0,
  min_sum_hessian_in_leaf = 0.001,
  lambda_l1            = 0.0,
  lambda_l2            = 0.0,
  bagging_fraction     = 1.0,
  pos_bagging_fraction = 1.0,
  neg_bagging_fraction = 1.0,
  is_unbalance         = FALSE,
  scale_pos_weight     = 1.0,
  drop_rate            = 0.1,
  max_drop             = 50,
  skip_drop            = 0.5,
  max_bin              = 31L,
  num_iterations       = 2185L,
  learning_rate        = 0.0376591746957654,
  feature_fraction     = 0.381721297354153,
  min_data_in_leaf     = 338L,   
  num_leaves           = 217L)
Sys.time()

In [ ]:
Sys.time()
PARAM$train_final$APO <- 6
PARAM$train_final$ksemillerio  <- 6
PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000)
Sys.time()

In [ ]:
Sys.time()
print("Este es el coso que genera semillas")
if(!require("primes")) install.packages("primes")
require("primes")
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]
cat(PARAM$BO$semillas)
Sys.time()

In [ ]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )]
PARAM$train_final$semillas

In [ ]:
Sys.time()
print("Acá uso dtrain_final que me parece que ya tiene los canaritos")
library(lightgbm)
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)
print("Este número debería de coincidir con: filas 265735 columnas 799")
cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [ ]:
Sys.time()
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}
Sys.time()

In [ ]:
Sys.time()
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]
Sys.time()

In [ ]:
mganancias <- matrix(nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes))
if( file.exists("prediccion1.txt") )
  file.remove("prediccion1.txt")